In [1]:
import jax.numpy as jnp
import jax
from tensorflow_probability.substrates import jax as tfp
import jax.scipy.special as jss
from functools import partial
import matplotlib.pyplot as plt
from jax.config import config
import numpy as np
import json
config.update("jax_enable_x64", True)

tfd = tfp.distributions
tfb = tfp.bijectors
tfpk = tfp.math.psd_kernels

import scipy.special as ss


import bridgestan as bs
import os
bs.set_bridgestan_path('/mnt/home/mjhajaria/.bridgestan/bridgestan-2.0.0')

import jaxtransforms

2023-11-03 06:38:22.892373: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 06:38:22.892410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 06:38:22.892439: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 06:38:23.819762: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_hessian(transform, datapath, y, frompackage=None):

    stan_filename=f'../stan_models/simplex/{transform}_DirichletSymmetric.stan'
    with open(stan_filename, 'w') as f:
        f.write(f'#include target_densities/DirichletSymmetric.stan{os.linesep}#include transforms/simplex/{transform}.stan{os.linesep}')
        f.close()

    model = bs.StanModel.from_stan_file(stan_filename, datapath,
                                        stanc_args=[f"--include-paths='/mnt/home/mjhajaria/transforms/'"])

    lp, grad, bshessian = model.log_density_hessian(y)
    print('bs hessian:', np.linalg.cond(bshessian), '\n\n bs eigenvalues:', np.linalg.eig(bshessian)[0], '\n\n')

    lp_stan_fn = getattr(jaxtransforms, f'{transform}_lp_stan')    
    hessian = jax.hessian(lp_stan_fn)(y, alpha, N)
    print('jax hessian:', jnp.linalg.cond(hessian), '\n\n jax eigenvalues:', jnp.linalg.eig(hessian)[0])
    
    globals()[f'{transform}_hessian_jax'] = hessian
    globals()[f'{transform}_hessian_bs'] = bshessian

In [3]:
N =  10
alpha = jnp.asarray([1]*N)
data = {"alpha": [1]*N, "N": N}
with open('data.json', 'w', encoding ='utf8') as json_file:
    json.dump(data, json_file, ensure_ascii = True)
datapath='data.json'

I0000 00:00:1699018705.200309 3635533 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
2023-11-03 06:38:25.664739: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:276] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [4]:
get_hessian('ALR', datapath, np.random.random(N-1))

bs hessian: 23.390109551352186 

 bs eigenvalues: [-0.07050418 -1.64910045 -1.28207084 -1.08306404 -0.70468863 -0.74999348
 -0.82219915 -0.94233875 -0.94955596] 


jax hessian: 23.390109566872283 

 jax eigenvalues: [-0.07050418+0.j -1.64910045+0.j -1.28207084+0.j -1.08306404+0.j
 -0.70468863+0.j -0.74999348+0.j -0.82219915+0.j -0.94233875+0.j
 -0.94955596+0.j]


In [5]:
ALR_hessian_jax

Array([[-0.7157296 ,  0.08839964,  0.05371808,  0.07339523,  0.07388856,
         0.05671663,  0.137954  ,  0.07291812,  0.10554447],
       [ 0.08839964, -1.00947045,  0.07887102,  0.10776179,  0.10848613,
         0.08327361,  0.20254955,  0.10706129,  0.15496459],
       [ 0.05371808,  0.07887102, -0.64437125,  0.06548394,  0.0659241 ,
         0.05060313,  0.12308391,  0.06505826,  0.09416781],
       [ 0.07339523,  0.10776179,  0.06548394, -0.85641998,  0.09007236,
         0.06913926,  0.16817004,  0.08888937,  0.12866186],
       [ 0.07388856,  0.10848613,  0.0659241 ,  0.09007236, -0.86157112,
         0.06960399,  0.16930042,  0.08948685,  0.12952668],
       [ 0.05671663,  0.08327361,  0.05060313,  0.06913926,  0.06960399,
        -0.67751542,  0.12995446,  0.06868982,  0.09942427],
       [ 0.137954  ,  0.20254955,  0.12308391,  0.16817004,  0.16930042,
         0.12995446, -1.46180774,  0.16707685,  0.24183341],
       [ 0.07291812,  0.10706129,  0.06505826,  0.08888937,  0

In [6]:
ALR_hessian_bs

array([[-0.7157296 ,  0.08839964,  0.05371808,  0.07339523,  0.07388856,
         0.05671663,  0.137954  ,  0.07291812,  0.10554447],
       [ 0.08839964, -1.00947045,  0.07887102,  0.10776179,  0.10848613,
         0.08327361,  0.20254955,  0.10706129,  0.15496459],
       [ 0.05371808,  0.07887102, -0.64437125,  0.06548394,  0.0659241 ,
         0.05060313,  0.12308391,  0.06505826,  0.09416781],
       [ 0.07339523,  0.10776179,  0.06548394, -0.85641998,  0.09007236,
         0.06913926,  0.16817004,  0.08888937,  0.12866186],
       [ 0.07388856,  0.10848613,  0.0659241 ,  0.09007236, -0.86157112,
         0.06960399,  0.16930042,  0.08948685,  0.12952668],
       [ 0.05671663,  0.08327361,  0.05060313,  0.06913926,  0.06960399,
        -0.67751542,  0.12995446,  0.06868982,  0.09942427],
       [ 0.137954  ,  0.20254955,  0.12308391,  0.16817004,  0.16930042,
         0.12995446, -1.46180774,  0.16707685,  0.24183341],
       [ 0.07291812,  0.10706129,  0.06505826,  0.08888937,  0

In [7]:
get_hessian('Stickbreaking', datapath, np.random.random(N-1))

bs hessian: 2.8916633561562564 

 bs eigenvalues: [-1.43019047 -1.18972086 -1.1562029  -0.8755574  -0.8803911  -0.88634996
 -0.73646496 -1.11808551 -0.49459093] 


jax hessian: 2.891663356065336 

 jax eigenvalues: [-1.43019047+0.j -1.18972086+0.j -1.11808551+0.j -1.1562029 +0.j
 -0.73646496+0.j -0.88634996+0.j -0.8755574 +0.j -0.8803911 +0.j
 -0.49459093+0.j]


In [8]:
get_hessian('HypersphericalProbit', datapath, np.random.random(N-1))

bs hessian: 1.0000000000091671 

 bs eigenvalues: [-1. -1. -1. -1. -1. -1. -1. -1. -1.] 


jax hessian: 1.0 

 jax eigenvalues: [-1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j]


In [9]:
get_hessian('HypersphericalLogit', datapath, np.random.random(N-1))

bs hessian: 1.1879781485728416 

 bs eigenvalues: [-0.4567075  -0.42876378 -0.41834376 -0.49429058 -0.47141576 -0.49698325
 -0.48063705 -0.43177185 -0.49013381] 


jax hessian: 1.18797814856812 

 jax eigenvalues: [-0.4567075 +0.j -0.42876378+0.j -0.41834376+0.j -0.49429058+0.j
 -0.47141576+0.j -0.49698325+0.j -0.48063705+0.j -0.43177185+0.j
 -0.49013381+0.j]


In [10]:
get_hessian('HypersphericalAngular', datapath, np.random.random(N-1))

bs hessian: 6.061541789748529 

 bs eigenvalues: [-4.31323097 -5.11383038 -4.77367467 -3.9031691  -2.67534395 -2.95363288
 -2.22183057 -1.67060359 -0.84365176] 


jax hessian: 6.061541789768432 

 jax eigenvalues: [-4.31323097+0.j -5.11383038+0.j -4.77367467+0.j -3.9031691 +0.j
 -2.67534395+0.j -2.95363288+0.j -2.22183057+0.j -1.67060359+0.j
 -0.84365176+0.j]


In [11]:
get_hessian('AugmentedSoftmax', datapath, np.random.random(N))

bs hessian: 15.27092372201248 

 bs eigenvalues: [-0.09881787 -1.50904012 -1.46643063 -1.36248067 -1.13977819 -0.96481137
 -0.89382292 -0.75322112 -0.67295846 -0.64891138] 


jax hessian: 15.270923737150085 

 jax eigenvalues: [-0.09881787+0.j -1.50904012+0.j -1.46643063+0.j -1.36248067+0.j
 -1.13977819+0.j -0.96481137+0.j -0.89382292+0.j -0.75322112+0.j
 -0.67295846+0.j -0.64891138+0.j]


In [12]:
get_hessian('ProbitProduct', datapath, np.random.random(N))

bs hessian: 5.060124140627858 

 bs eigenvalues: [-5.06012414 -4.9172117  -4.72759686 -3.76494606 -1.44918624 -2.89763606
 -1.98930438 -2.51447642 -1.        ] 


jax hessian: inf 

 jax eigenvalues: [-5.06012414+0.j -4.9172117 +0.j -4.72759686+0.j -3.76494606+0.j
 -1.44918624+0.j -1.98930438+0.j -2.89763606+0.j -2.51447642+0.j
 -1.        +0.j  0.        +0.j]


In [13]:
get_hessian('NormalizedExponential', datapath, np.random.random(N))

bs hessian: 1.0000000000091671 

 bs eigenvalues: [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1.] 


jax hessian: inf 

 jax eigenvalues: [-1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j -1.+0.j
  0.+0.j]


In [14]:
get_hessian('AugmentedILR', datapath, np.random.random(N-1))

bs hessian: 6.285521186766866 

 bs eigenvalues: [-0.26781211 -1.68333872 -1.43034983 -1.30278736 -1.12014077 -0.89678611
 -0.65717117 -0.68999579 -0.75646341] 


jax hessian: 6.285521186934499 

 jax eigenvalues: [-0.26781211+0.j -1.68333872+0.j -1.43034983+0.j -1.30278736+0.j
 -1.12014077+0.j -0.89678611+0.j -0.65717117+0.j -0.68999579+0.j
 -0.75646341+0.j]
